# Extraccion, transformacion y carga de datos

### Importamos librerias

In [1]:
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from modulos.data.webdriver import iniciar_chrome
from modulos.utils.wait import *
import pandas as pd
import re
import warnings

warnings.filterwarnings('ignore') # Desactiva las advertencias en la caja de resultados

In [129]:
# Para recargar las librerias locales
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Verifiquemos el sitio web


Definimos url de la pagina a recabar informacion, y el user agent (requisito)

In [3]:
url = "https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderDirection=DESC&OrderColumn=Price&PagingPageSkip=0"
h = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}

Verificamos el funcionamiento


In [4]:
try:
    res = requests.get(url=url,headers=h, timeout= 10)
except TimeoutError:
    print("La pagina no arroja resultados")

In [5]:

if res.status_code <400 and res.ok == True:
    print("La solicitud ha sido aprovada con exito")
else:
    print(f"Ha fallado\n codigo:{res.status_code}")

La solicitud ha sido aprovada con exito


Abrimos el codigo html para comprobar si estamos ubicados donde deseamos

In [6]:
with open("../references/codigo_200.html","w",encoding="utf-8") as f:
    f.write(res.text)

Al hacer Preview al archivo html podemos ver que estamos bien ubicados
![image.png](img/PreviewHTML.png)

## Extraccion

Iniciamos el bucle "for" para recolectar todas las paginas de oferta del portal, esto podemos hacerlo creando una lista con las url de las 41 paginas de ofertas 



In [7]:
# URL base con el marcador {page} donde se insertará el número de página
url_base = "https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip={page}"
h = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}
# Lista para almacenar las URLs
ofertas = []

# Bucle para crear las URLs cambiando el valor de 'PagingPageSkip' de 0 a 41
for i in range(42):
    # Reemplaza {page} con el valor de i
    url = url_base.format(page=i)
    # Agrega la URL generada a la lista
    ofertas.append(url)

# Imprime la lista de URLs
print(ofertas)

['https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=0', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=1', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=2', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=3', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=4', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401

In [8]:
# Lista para guardar los enlaces de las ofertas
enlaces_ofertas = []

# Bucle para recorrer cada URL
for url in ofertas:
    # Realiza una solicitud GET a la URL
    response = requests.get(url, headers=h)
    pausa()
    # Verifica si la solicitud fue exitosa
    if response.status_code < 300:
        # Analiza el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encuentra el contenedor de resultados
        contenedor_resultados = soup.find('div', id='bigsearch-results-inner-results')
        
        if contenedor_resultados:
            # Encuentra todas las etiquetas <li> dentro del contenedor
            ofertas = contenedor_resultados.find_all('li')
            # Recorre cada oferta y extrae el enlace del bloque <a>
            for oferta in ofertas:
                enlace = oferta.find('a', href=True)
                if enlace:
                    # Obtiene el atributo href y lo agrega a la lista
                    url_completo = "https://supercasas.com" + enlace['href']
                    enlaces_ofertas.append(url_completo)
                    



In [9]:
# Imprime la lista de enlaces de las ofertas
print(enlaces_ofertas[:3]) # observamos que tenemos los enlaces funcionales, primeras 3



['https://supercasas.com/apartamentos-alquiler-el-millon/1367994/', 'https://supercasas.com/apartamentos-venta-y-alquiler-av--anacaona/1358370/', 'https://supercasas.com/apartamentos-venta-y-alquiler-av--anacaona/1357799/']


In [10]:
print(enlaces_ofertas[-3:]) #ultimas 3

['https://supercasas.com/apartamentos-alquiler-centro-de-los-heroes/1366244/', 'https://supercasas.com/apartamentos-alquiler-30-de-mayo/1366238/', 'https://supercasas.com/apartamentos-alquiler-evaristo-morales/1366232/']


Comprobemos el tamaño que deberia ser la lista para verificar si no hay inciertos

Segun calculos manuales si cada pagina contiene 24 ofertas y existen 42 paginas, tenemos que es 24 (ofertas) * 41 (paginas) + 16 que contiene la ultima pagina = 1000 ofertas

In [11]:
#Verifiquemos si enlaces_ofertas contiene 1000 elementos

if len(enlaces_ofertas) == 1000:
    print("Si tiene 1000 ofertas")
else:
    print(f"No tiene 1000 ofertas, tiene {len(enlaces_ofertas)} ofertas")

Si tiene 1000 ofertas


Ya tenemos extraido cada oferta de la pagina, creemos un dataset con la informacion especifica de cada oferta:
metros cuadrados, sector, cantidad de baños, habitacion y parqueos, 

Variables a extraer:

* precio
* sector
* habitaciones
* baños (si es decimal, ej: x.5 , significara x baños y 1 medio baño)
* parqueos
* Condicion
* Metros cuadrados (construccion)
* nivel/piso (si es 0 sera considerado N/A)
* Uso actual 
* Terreno (si es 0 sera considerado N/A)
* ascensores (verificar si en comodidades aparece en caso de decir 0, solo se tomara en cuenta 1 si posee 0 si no)
* Edificable (Si o No)
* año de construccion (N/D es considerado N/A)
* Posee planta electrica
* Posee seguridad 24 horas
* Posee control de Acceso
* Posee piscina(palabras similares como jacuzzi y picuzzi seran considerados como piscina tambien)
* Posee gimnasio (Gym o gimnasio)
  

In [12]:
#Ejemplo de sacar precio
response =requests.get(enlaces_ofertas[2],headers=h)
if response.status_code <300:
    soup = BeautifulSoup(response.text, 'html.parser')
    precio = soup.find('div', class_="detail-ad-info-specs-block main-info").find('span', text= re.compile(r'^Alquiler')).find_parent('div').text 
    moneda = float(precio.split(" ")[-1].split("/")[0].replace(",",""))
    # soup.find busca en el selector la palabra Alquiler y la primera que encuentra extrae el precio. Si aparece "alquiler" y "alquiler amueblado" cogera el precio del alquiler e ignora el amueblado
    pesos_a_dolar = 60 # tasa de cambio 1 dolar es 60 pesos
    
    if "US$" in precio:
        dolares = moneda
        print(dolares)
    elif "RD$" in precio:
        dolares = float(moneda/pesos_a_dolar) # transformamos pesos  adolares
        print(dolares)
    else:
        dolares = None


    

11000.0


In [17]:
#Ejemplo de sacar sector
response = requests.get(enlaces_ofertas[7],headers=h)
if response.status_code <300:
    soup = BeautifulSoup(response.text, 'html.parser')
    #Sacar cantidad de habitaciones, baños y parqueos
    cantidad = soup.find("div",class_= "detail-ad-info-specs-block secondary-info").find_all("div")
    habitaciones = int(cantidad[0].text.split(" ")[0])
    bagnos = float(cantidad[1].text.split(" ")[0])
    parqueos = int(cantidad[2].text.split(" ")[0])

    #print(parqueos)

    #Sacar contenido de la tabla de 'Datos Generales'
    tabla = soup.find("div", id="detail-ad-info-specs").find("table")

    sector = tabla.find('td',colspan = "3").text.split(">")[-1]
    condicion = soup.find('td', text=lambda text: text and "Condición:" in text).find_parent('tr').find_all("td")[1].text
    uso_actual = soup.find('td', text=lambda text: text and "Condición:" in text).find_parent('tr').find_all("td")[3].text
    metrosC = float(soup.find('td', text=lambda text: text and "Terreno:" in text).find_parent('tr').find_all("td")[1].string.split(" ")[0])
    terreno = float(soup.find('td', text=lambda text: text and "Terreno:" in text).find_parent('tr').find_all("td")[3].string.split(" ")[0])
    piso = int(soup.find('td', text=lambda text: text and "Ascensores:" in text).find_parent('tr').find_all("td")[1].text)

    ascensores = int(soup.find('td', text=lambda text: text and "Ascensores:" in text).find_parent('tr').find_all("td")[3].text)
    ascensores = ascensores >=1

    edificable = soup.find('td', text=lambda text: text and "Edificable:" in text).find_parent('tr').find_all("td")[1].text

    agno = soup.find('td', text=lambda text: text and "Edificable:" in text).find_parent('tr').find_all("td")[3].text
    if agno.isdigit():
        agno = int(agno)

    #print(agno)

    #Extraer en la tabla comodidades

    planta_electrica = seguridad = control_Acceso = piscina = gimnasio = False
    try:
        comodidades = soup.find('h3', text=lambda text: text and "Comodidades:" in text).find_parent('div').find_all("li")
        
    
        
        #print(f"ascensor es {ascensores}")
        for elemento in comodidades:
            if "Planta Eléctrica" in elemento:
                planta_electrica = True
            elif "Seguridad 24 Horas" in elemento:
                seguridad = True 
            elif "Control de Acceso" in elemento:
                control_Acceso = True
            elif "Piscina" in elemento or "Jacuzzi" in elemento or "Picuzzi" in elemento:
                piscina = True
            elif "Gimnasio" in elemento or "Gym" in elemento:
                gimnasio = True
            elif "Ascensor" in elemento:
                ascensores = True
    except:
        pass
        
    try:
        observaciones = soup.find('h3', text=lambda text: text and "Observaciones:" in text).find_parent('div').find("p").text.lower()
        
        if "planta eléctrica" in observaciones:
            planta_electrica = True
        elif "seguridad 24 horas" in observaciones:
            seguridad = True 
        elif "control de acceso" in observaciones:
            control_Acceso = True
        elif "piscina" in observaciones or "jacuzzi" in observaciones or "picuzzi" in observaciones:
            piscina = True
        elif "gimnasio" in observaciones or "gym" in observaciones:
            gimnasio = True
        elif "ascensor" in observaciones or "ascensores" in observaciones:
            ascensores = True


    except:
        pass


    #lista = [planta_electrica,seguridad,control_Acceso,piscina,gimnasio]

    #print(f"ahora ascensor es {ascensores}")
    #print(lista)






In [22]:
# Configurar un adaptador de reintentos
session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504, 10054])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

In [23]:
# Lista para almacenar los datos de cada oferta
data = []

# Bucle para recorrer cada enlace de la oferta
for enlace in enlaces_ofertas:
    # Realiza una solicitud GET al enlace de la oferta
    pausa(0.5,2)
    try:
        # Realiza una solicitud GET al enlace de la oferta con la sesión configurada para reintentos
        response = session.get(enlace, headers=h, timeout=10)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa
    except requests.exceptions.RequestException as e:
        print(f"Error al acceder a {enlace}: {e}")
        continue  # Si hay un error, pasar al siguiente enlace


    # Verifica si la solicitud fue exitosa
    if response.status_code < 300:
        # Analiza el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')
        
        try:
            # Extraer Precio
            precio = soup.find('div', class_="detail-ad-info-specs-block main-info").find('span', text= re.compile(r'^Alquiler')).find_parent('div').text 
            moneda = float(precio.split(" ")[-1].split("/")[0].replace(",",""))
            # soup.find busca en el selector la palabra Alquiler y la primera que encuentra extrae el precio. Si aparece "alquiler" y "alquiler amueblado" cogera el precio del alquiler e ignora el amueblado
            pesos_a_dolar = 60 # tasa de cambio 1 dolar es 60 pesos
        
            if "US$" in precio:
                dolares = moneda
            elif "RD$" in precio:
                dolares = float(moneda/pesos_a_dolar) # transformamos pesos  adolares
            else:
                dolares = None




            # Extraer cantidad de habitaciones, baños y parqueos
            cantidad = soup.find("div",class_= "detail-ad-info-specs-block secondary-info").find_all("div")
            habitaciones = int(cantidad[0].text.split(" ")[0])
            bagnos = float(cantidad[1].text.split(" ")[0])
            parqueos = int(cantidad[2].text.split(" ")[0])
        except:
            pass
        
        try:
            # Extraer contenido de la tabla de 'Datos Generales'
            tabla = soup.find("div", id="detail-ad-info-specs").find("table")

            sector = tabla.find('td',colspan = "3").text.split(">")[-1]
            condicion = soup.find('td', text=lambda text: text and "Condición:" in text).find_parent('tr').find_all("td")[1].text
            uso_actual = soup.find('td', text=lambda text: text and "Condición:" in text).find_parent('tr').find_all("td")[3].text
            metrosC = float(soup.find('td', text=lambda text: text and "Terreno:" in text).find_parent('tr').find_all("td")[1].string.split(" ")[0])
            terreno = float(soup.find('td', text=lambda text: text and "Terreno:" in text).find_parent('tr').find_all("td")[3].string.split(" ")[0])
            piso = int(soup.find('td', text=lambda text: text and "Ascensores:" in text).find_parent('tr').find_all("td")[1].text)

            ascensores = int(soup.find('td', text=lambda text: text and "Ascensores:" in text).find_parent('tr').find_all("td")[3].text)
            ascensores = ascensores >=1

            edificable = soup.find('td', text=lambda text: text and "Edificable:" in text).find_parent('tr').find_all("td")[1].text

            agno = soup.find('td', text=lambda text: text and "Edificable:" in text).find_parent('tr').find_all("td")[3].text
            if agno.isdigit():
                agno = int(agno)

        except:
            pass
        
        # Extraer en la tabla comodidades

        planta_electrica = seguridad = control_Acceso = piscina = gimnasio = False

        try:
            comodidades = soup.find('h3', text=lambda text: text and "Comodidades:" in text).find_parent('div').find_all("li")
        
            
        
            for elemento in comodidades:
                if "Planta Eléctrica" in elemento:
                    planta_electrica = True
                elif "Seguridad 24 Horas" in elemento:
                    seguridad = True 
                elif "Control de Acceso" in elemento:
                    control_Acceso = True
                elif "Piscina" in elemento or "Jacuzzi" in elemento or "Picuzzi" in elemento:
                    piscina = True
                elif "Gimnasio" in elemento or "Gym" in elemento:
                    gimnasio = True
                elif "Ascensor" in elemento and ascensores == False:
                    ascensores = True
        except:
            pass
        
        try:
            observaciones = soup.find('h3', text=lambda text: text and "Observaciones:" in text).find_parent('div').find("p").text.lower()
            
            if "planta eléctrica" in observaciones:
                planta_electrica = True
            elif "seguridad 24 horas" in observaciones:
                seguridad = True 
            elif "control de acceso" in observaciones:
                control_Acceso = True
            elif "piscina" in observaciones or "jacuzzi" in observaciones or "picuzzi" in observaciones:
                piscina = True
            elif "gimnasio" in observaciones or "gym" in observaciones:
                gimnasio = True
            elif "ascensor" in observaciones or "ascensores" in observaciones:
                ascensores = True


        except:
            pass


        # Añade la información extraída como un registro en el dataset
        data.append({
            'Precio': dolares,
            'Habitaciones': habitaciones,
            'Baños': bagnos,
            'Parqueos': parqueos,
            'Sector': sector,
            'Condicion': condicion,
            'Uso Actual': uso_actual,
            'mt2': metrosC,
            'Terreno mt2': terreno,
            'Piso/Nivel': piso,
            'Ascensor': ascensores,
            'Edificable': edificable,
            'Agno Construccion': agno,
            'Planta Electrica': planta_electrica,
            'Seguridad 24 Horas': seguridad,
            'Control de Acceso': control_Acceso,
            'Piscina': piscina,
            'Gimnasio': gimnasio 
            
        })

df = pd.DataFrame(data)
df.to_csv('../data/raw/ofertas_inmobiliarias.csv', index=False, encoding='utf-8')